### Mounting Azure Storage Account/conatiners using Service Principal and AZ KeyVault

Steps to that have been prepared:
- creating an Azure KeyVault
- creating Secret Scope to link Databricks to KV
- creating a Service Principal
- setting all the secrets


In [0]:
# List the secrets in specific scope
display(dbutils.secrets.list("scope-ennatuurlijk"))

In [0]:
# Define DataLakeMounter function

def DataLakeMounter(storage_account_name, container_names):
    # Set secret scope
    secret_scope = "scope-ennatuurlijk"

    # Set secrets & vars
    client_id = dbutils.secrets.get(scope=secret_scope,key="ennatuurlijk-client-id")
    tenant_id = dbutils.secrets.get(scope=secret_scope,key="tenant-id")
    client_secret = dbutils.secrets.get(scope=secret_scope,key="ennatuurlijk-client-secret")
    sa_endpoint = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

    # Standard spark configs to be used (from MS documentation)
    configs = {
        "fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": client_id,
        "fs.azure.account.oauth2.client.secret": client_secret,
        "fs.azure.account.oauth2.client.endpoint": sa_endpoint
    }

    # Check if mount already exists before attempting mount
    for container_name in container_names:
        mount_path = f"/mnt/{storage_account_name}/{container_name}"
        existing_mounts = [mount.mountPoint for mount in dbutils.fs.mounts()]
        
        # If mount exists, unmount it to make sure mounting does not fail
        if mount_path in existing_mounts:
            dbutils.fs.unmount(mount_path)

        # Do the actual mounting
        dbutils.fs.mount(
            source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
            mount_point=mount_path,
            extra_configs=configs
        )
    
    display(dbutils.fs.mounts())

In [0]:
# Mount containers into Databricks

storage_account_name = dbutils.secrets.get(scope="scope-ennatuurlijk",key="ennatuurlijk-sa-name")
container_names = [
    "bronze",
    "silver",
    "gold"
]

DataLakeMounter(storage_account_name, container_names)
